In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Getting the Data

In [ ]:
df_train=pd.read_csv('../input/hackerearths-reduce-marketing-waste/train.csv')
df_sample=pd.read_csv('../input/hackerearths-reduce-marketing-waste/sample_submission.csv')
df_test=pd.read_csv('../input/hackerearths-reduce-marketing-waste/test.csv')

# EDA

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
plt.figure(figsize=(13,5))
sns.heatmap(pd.isnull(df_train))

So we can see that some of the columns have null values!!

In [ ]:
df_train.columns

In [ ]:
df_train.head(4)

Columns to be dropped "Industry" to many categoies in industry columns.
                      "Location" irrelevent
                      "Hiring_candidate_role" 
                      "Internal POC"
                      
Null values to be handled in - Geography ,Last_lead_update,Resource

In [ ]:
fig, axs = plt.subplots(ncols=3,figsize=(15,5))
sns.countplot(df_train['Pitch'], ax=axs[0])
sns.countplot(df_train['Lead_revenue'], ax=axs[1])
sns.countplot(df_train['Fund_category'] ,ax=axs[2])


In [ ]:
fig, axs = plt.subplots(ncols=3,figsize=(15,5))
sns.countplot(df_train['Geography'], ax=axs[0])
sns.countplot(df_train['Designation'], ax=axs[1])
sns.countplot(df_train['Lead_source'] ,ax=axs[2])


In [ ]:
fig, axs = plt.subplots(ncols=3,figsize=(15,5))
sns.countplot(df_train['Level_of_meeting'], ax=axs[0])
sns.countplot(df_train['Last_lead_update'], ax=axs[1])
sns.countplot(df_train['Resource'] ,ax=axs[2])

In [ ]:
df=df_train.drop(['Industry','Location','Hiring_candidate_role','Internal_POC'],axis=1)

In [ ]:
df.shape

# Dealing with null Values

Replace values in Last_lead_update,Resource 

In [ ]:
def nvalues(p):
    c=p
    if pd.isna(p)==True:
        c="No track"
    return c
def nvalues2(p):
    c=p
    if pd.isna(p)==True:
        c="No"
    return c
        

In [ ]:
df['Last_lead_update']=df['Last_lead_update'].apply(nvalues)

In [ ]:
df['Resource']=df['Resource'].apply(nvalues2)

In [ ]:
plt.figure(figsize=(13,5))
sns.heatmap(pd.isnull(df))

# Checking Correlation of Deal value and weight amount

In [ ]:
df[df['Deal_value'] == 'na$']

In [ ]:
df.drop(index, inplace=True)

In [ ]:
def deal(a):
    a=str(a)
    b=a[:-1]
    return b
def deal2(c):
    return int(c)
def deal3(e):
    f=str(e)
    g=f[:-1]
    return float(g)
    

In [ ]:
deal("320506$")

In [ ]:
deal(df['Deal_value'][3])

In [ ]:
df['Deal_value']=df['Deal_value'].apply(deal)

In [ ]:
df=df[df['Deal_value']!='na']

In [ ]:
df['Deal_value']=df['Deal_value'].apply(deal2)

In [ ]:
df_1=df.dropna(axis=0)

In [ ]:
sns.heatmap(df_1.isnull())

In [ ]:
df_1['Weighted_amount']=df_1['Weighted_amount'].apply(deal3)

In [ ]:
np.corrcoef(df_1['Weighted_amount'],df_1['Deal_value'])

In [ ]:
df_2=df.dropna(axis=1)

In [ ]:
sns.heatmap(df_2.isnull())

In [ ]:
df_2.head()

In [ ]:
df_2=df_2.drop(['Lead_name','Date_of_creation','Contact_no'],axis=1)

In [ ]:
df_2.shape

'Lead_name'    'Date_of_creation' 'Contact_no'

In [ ]:
df_2

In [ ]:
def domain(p):
    test_str=str(p)
    res = test_str[test_str.index('@') + 1 : ]
    return res

In [ ]:
df_2['Domain']=df_2['Lead_POC_email'].apply(domain)

In [ ]:
df_2=df_2.drop(['Domain','Lead_POC_email','Deal_title'],axis=1)

In [ ]:
df_2.head()

In [ ]:
df_2.columns

In [ ]:
df_3=pd.get_dummies(data=df_2,columns={'Pitch', 'Lead_revenue', 'Fund_category',
       'Designation', 'Lead_source', 'Level_of_meeting', 'Last_lead_update',
       'Resource', 'Internal_rating'})

In [ ]:
df_3.shape

In [ ]:
X=df_3.drop('Success_probability',axis=1)

In [ ]:
y=df_3['Success_probability']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8,test_size=0.2, random_state=101)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler=MinMaxScaler()

In [ ]:
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
model_linear=LinearRegression()

In [ ]:
model_linear.fit(X_train,y_train)

In [ ]:
y_pred_lin=model_linear.predict(X_test)
y_train_lin=model_linear.predict(X_train)

In [ ]:
mean_squared_error(y_train, y_train_lin, squared=False)

In [ ]:
mean_squared_error(y_test, y_pred, squared=False)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model_ran=RandomForestRegressor(n_estimators=30)

In [ ]:
model_ran.fit(X_train,y_train)

In [ ]:
y_train_ran=model_ran.predict(X_train)

In [ ]:
mean_squared_error(y_train, y_train_ran, squared=False)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model_1 = Sequential()
model_1.add(Dense(49, activation='relu'))

#Dropout(0.2)

#Dropout(0.2)
model_1.add(Dense(25, activation='relu'))

#Dropout(0.2)

model_1.add(Dense(12, activation='relu')) 
#Dropout(0.2)
model_1.add(Dense(6, activation='relu'))
model_1.add(Dense(6, activation='relu'))
#Dropout(0.2)

model_1.add(Dense(4, activation='relu'))
model_1.add(Dense(2, activation='relu'))
model_1.add(Dense(1, activation='relu'))
model_1.compile(loss='mse', optimizer='adam')

In [ ]:
early=EarlyStopping(monitor='loss',patience=30,verbose=0,mode='max')

In [ ]:
model_1.fit(X_train, y_train, epochs=200,callbacks=[early],validation_data=(X_test, y_test))